In [1]:
import re
import contractions
import jieba
import pandas as pd
import nltk
import pickle
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from zhconv import convert
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
jieba.load_userdict('./jieba/dict_big.txt')


def scToTc(text):
    text = convert(text, 'zh-tw')

    return text


def expandContraction(text):
    # specific
    text = re.sub(r'i[\'?]m', 'i am', text)
    text = re.sub(r'let[\'?]s', 'let us', text)
    text = re.sub(r'don[\'?]t', 'do not', text)
    text = re.sub(r'can[\'?]t', 'can not', text)
    text = re.sub(r'won[\'?]t', 'will not', text)

    # general
    text = re.sub(r'[\'?]s', ' is', text)
    text = re.sub(r'[\'?]re', ' are', text)
    text = re.sub(r'[\'?]ll', ' will', text)
    text = re.sub(r'[\'?]d', ' would', text)
    text = re.sub(r'[\'?]ve', ' have', text)
    text = re.sub(r'n[\'?]t', ' not', text)

    # library
    text = contractions.fix(text)

    return text


def cleanData(text):
    # expand contraction
    text = expandContraction(text)

    # replace hyperlink
    text = re.sub(r'http[s]?:\/\/[\w\/.?=-]+', ' link ', text)

    # replace email address
    text = re.sub(r'[\w\.+]+@[\w\.]+\.[a-z]{2,}', ' email ', text)

    # replace currency sign
    text = re.sub(r'[\$€£¥]', ' money ', text)

    # replace number
    text = re.sub(r'[\d]+', ' number ', text)

    # replace special char, other than a-z, A-Z, 0-9 and chinese
    text = re.sub(r'[^a-zA-Z0-9\u4E00-\u9FFF]+', ' ', text)

    # replace new line (carriage return and line feed)
    text = re.sub(r'[\r\n]', ' ', text)

    # replace white space
    text = re.sub(r'[\s]{2,}', ' ', text)
    text = re.sub(r'^[\s]+|[\s]+$', '', text)

    return text


def stopWords(text, words):
    text = ' '.join([word for word in text.split() if word not in (words)])

    return text


def stemming(text, stemmer):
    text = ' '.join([stemmer.stem(word) for word in text.split()])

    return text


def lemmatization(text, lemmatizer):
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

    return text


def segmentation(text):
    text = ' '.join(jieba.cut(text))

    return text


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\user\AppData\Local\Temp\jieba.cache
Loading model cost 1.078 seconds.
Prefix dict has been built successfully.


In [2]:
input = ['[Netflix] : 無法處理你的自動付款。你的帳戶將被禁用。t.co/ntAkhpFWqR',
         '亲爱的CHANKWAN POK先生或女士，欢迎您加入“心享”计划 HPR”，您的会员编码为：111023621183，初始密码为：006653。恭喜您可尊享通过“中旅酒店心享会”公众号推荐朋友加入会员活动，被推荐的新会员还可以领取代金券礼包！详询400-669-0000（大陆地区）、852-36040000（港澳地区）。',
         '《天外》全新資料片登場!開放亞特蘭提斯城!推出英雄升變!英雄能力大解放! mo.alta.hk 查詢EN/取消UN81060822',
         '渣打香港: 您現正使用尾數為0376之信用卡在Pure International HK L進行一項網上交易，金額HKD 2,038.00;你的一次有效密碼為NGY-730596。',
         '''您好！我系橙橙
邀請您加细妹微信：76169639 睇朋友圈保有您喜歡嘅哦！
有國際大牌：L.V、愛馬仕、普拉達、迪奧、聖羅蘭、古奇、芬迪、巴寶莉、阿瑪尼、寶格麗、勞力士等（男女服裝、鞋子、包包、手錶、皮帶、圍巾、首飾等）第三方担保交易，驗貨滿意再签收，購物零風險
我要告訴您：我賣嘅系超A貨，我沒有去欺騙任何人，嗰啲揾我買嘅人，他們都很清楚''']

# 1, 1, 0, 0, 1


def preprocess(input, remove_stop, stem, lemmatize):
    words = stopwords.words('english')
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()

    output = []

    for text in input:
        text = str(text)
        text = text.lower()
        text = scToTc(text)
        text = cleanData(text)

        if remove_stop:
            text = stopWords(text, words)

        if stem:
            text = stemming(text, stemmer)

        if lemmatize:
            text = lemmatization(text, lemmatizer)

        text = segmentation(text)
        text = re.sub(r'[\s]{2,}', ' ', text)

        output.append(text)
    return output


output = preprocess(input, True, True, False)

print(output)


['netflix 無法 處理 你 的 自動 付款 你 的 帳戶 將被 禁用 co ntakhpfwqr', '親愛 的 chankwan pok 先生 或 女士 歡迎您 加入 心享 計劃 hpr 您 的 會員 編碼 為 number 初始密碼 為 number 恭喜 您 可 尊享 通過 中旅 酒店 心享 會 公眾號 推薦 朋友 加入 會員 活動 被 推薦 的 新會員 還 可以 領取 代金券 禮包 詳詢 number number number 大陸 地區 number number 港澳地區', '天外 全新 資料片 登場 開放 亞特蘭 提 斯城 推出 英雄 升變 英雄 能力 大 解放 mo alta hk 查詢 en 取消 un number', '渣打 香港 您 現正 使用 尾數 為 number 之 信用卡 在 pure intern hk l 進行 一項 網上交易 金額 hkd number number number 你 的 一次 有效 密碼 為 ngi number', '您好 我系 橙 橙 邀請 您 加細妹 微信 number 睇 朋友圈 保有 您 喜歡 嘅 哦 有 國際 大牌 l v 愛馬仕 普拉達 迪奧 聖羅蘭 古奇 芬迪 巴寶莉 阿瑪尼 寶格麗 勞力士 等 男女 服裝 鞋子 包包 手錶 皮帶 圍巾 首飾 等 第三方 擔保 交易 驗貨 滿意 再 籤 收 購物 零 風險 我要 告訴您 我 賣 嘅 系 超 a 貨 我 沒有 去 欺騙 任何人 嗰 啲 搵 我 買 嘅 人 他們 都 很 清楚']


In [3]:
def featureExtraction(input):
    tokenizer = pickle.load(open('./best_tokenizer.pkl', 'rb'))

    print(len(tokenizer.word_index))

    output = input.copy()
    output = tokenizer.texts_to_sequences(output)
    output = pad_sequences(output, maxlen=100, padding='post', truncating='post')

    return output


output = featureExtraction(output)

print(output.shape)
print(output)


45314
(5, 100)
[[17164   155   493     6     1   268   187     6     1   170 26317 17229
   1184 26318     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0]
 [  409     1 26408 26409  1443   203  2188  1391   304 26410  1039 26411
     14     1    35  2767   114     2 17267   114     2   822    14    15
   1169   202 13434   845 26410   110   190    73   439   304    35    80
    204    73     1  3694   129    13   137  1551   711   415     2     2
      2  3911  1221     2     2 26412     0     0     0     0     0   

In [4]:
loaded_model = load_model('./best_model.h5')
loaded_model.summary()

pred = loaded_model.predict(output)
true = [1, 1, 0, 0, 1]

print(pred)

score = accuracy_score(true, pred > 0.5)

print(score)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 32)           1450080   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 1,466,785
Trainable params: 1,466,785
Non-trainable params: 0
_________________________________________________________________
1/1 [==============================] - 1s 1s/step
[[0.9818954 ]
 [0.9998923 ]
 [0.01482893]
 [0.99958897]
 [0.99

* set01 = 0.9904858987427795
* set02 = 0.9459459459459459
* set03 = 0.9977263038226517
* set04 = 0.9862816336757705
